In [ ]:
!wget https://www2.eecs.berkeley.edu/Research/Projects/CS/vision/grouping/segbench/BSDS300-images.tgz
!wget https://www2.eecs.berkeley.edu/Research/Projects/CS/vision/grouping/segbench/BSDS300-human.tgz
!tar zxvf *-images.tgz
!tar zxvf *-human.tgz

In [ ]:
import re
import os.path
from PIL import Image
import numpy as np

In [ ]:
root_dir = "./BSDS300"
train_data = os.path.join(root_dir, 'images/train')
test_data = os.path.join(root_dir, 'images/test')
labels = os.path.join(root_dir, 'human/color')
humans = [folder for folder in os.listdir(labels) if not folder.startswith('.')]

In [ ]:
def extract_labels(label_files):
    """Extracts the segmentation data from the .seg files"""
    meta = {}
    data = []
    with open(label_files, 'r') as f:
        matcher = re.compile('(?P<seg>^[0-9 ]+)')
        for line in f:
            seg_match = matcher.search(line)
            if seg_match:
                string_segment = seg_match.group('seg').split(' ')
                int_segment = np.asarray(string_segment, dtype=int)
                data.append(int_segment)
                continue
            elif "data" not in line:
                meta_data = line.strip('\n').split(' ', 1)
                index, value = meta_data[0], meta_data[1]
                meta[index] = value
    height, width = int(meta['height']), int(meta['width'])
    seg_num = int(meta['segments'])
    # print(f"User id: {meta['user']}     Image id: {meta['image']}")
    # print(f"Height: {height}       Width: {width}")
    segmentation = np.zeros((height, width))
    for seg in data:
        segmentation[seg[1], seg[2]:(seg[3]+1)] = seg[0]
    return segmentation, seg_num

In [ ]:
def create_image(segmentation, seg_num):
    """Creates an image using extraced segmentation data"""
    segmentation = (segmentation / seg_num) * 255
    img = Image.fromarray(segmentation)
    img.show()

In [ ]:
for folder in humans:
    human_path = os.path.join(labels, folder)
    seg_files = [seg for seg in os.listdir(human_path)]
    targets = []
    for seg in seg_files:
        seg_path = os.path.join(human_path, seg)
        targets.append(extract_labels(seg_path))


In [ ]:
test = os.path.join(labels, '1115/66053.seg')
segmentations, seg_num = extract_labels(test)
create_image(segmentations, seg_num)
